In [21]:
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [22]:
train = pd.read_json('train.json')
test = pd.read_json('test.json')
train.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],medium,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",low,40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue
100004,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,"[Laundry In Building, Dishwasher, Hardwood Flo...",high,40.7388,6887163,-74.0018,d9039c43983f6e564b1482b273bd7b01,[https://photos.renthop.com/2/6887163_de85c427...,2850,241 W 13 Street
100007,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,"[Hardwood Floors, No Fee]",low,40.7539,6888711,-73.9677,1067e078446a7897d2da493d2f741316,[https://photos.renthop.com/2/6888711_6e660cee...,3275,333 East 49th Street
100013,1.0,4,0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,[Pre-War],low,40.8241,6934781,-73.9493,98e13ad4b495b9613cef886d79a6291f,[https://photos.renthop.com/2/6934781_1fa4b41a...,3350,500 West 143rd Street


In [23]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49352 entries, 10 to 99994
Data columns (total 15 columns):
bathrooms          49352 non-null float64
bedrooms           49352 non-null int64
building_id        49352 non-null object
created            49352 non-null object
description        49352 non-null object
display_address    49352 non-null object
features           49352 non-null object
interest_level     49352 non-null object
latitude           49352 non-null float64
listing_id         49352 non-null int64
longitude          49352 non-null float64
manager_id         49352 non-null object
photos             49352 non-null object
price              49352 non-null int64
street_address     49352 non-null object
dtypes: float64(3), int64(3), object(9)
memory usage: 6.0+ MB


In [24]:
#delete some object features

del train['building_id']
del train['created']
del train['description']
del train['display_address']
del train['manager_id']
del train['street_address']
del test['building_id']
del test['created']
del test['description']
del test['display_address']
del test['manager_id']
del test['street_address']

In [25]:
#convert remaining object features to numeric values

train['num_features'] = train['features'].apply(len)
train['num_photos'] = train['features'].apply(len)
test['num_features'] = test['features'].apply(len)
test['num_photos'] = test['features'].apply(len)
del train['features']
del train['photos']
del test['features']
del test['photos']
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49352 entries, 10 to 99994
Data columns (total 9 columns):
bathrooms         49352 non-null float64
bedrooms          49352 non-null int64
interest_level    49352 non-null object
latitude          49352 non-null float64
listing_id        49352 non-null int64
longitude         49352 non-null float64
price             49352 non-null int64
num_features      49352 non-null int64
num_photos        49352 non-null int64
dtypes: float64(3), int64(5), object(1)
memory usage: 3.8+ MB


In [26]:
#null accuracy

train['interest_level'].value_counts() / train.shape[0]

low       0.694683
medium    0.227529
high      0.077788
Name: interest_level, dtype: float64

In [27]:
#train test split

X, y = train.drop('interest_level', axis = 1), train['interest_level']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [28]:
#random forest

rand_forest = RandomForestClassifier(n_estimators = 100)
rand_forest.fit(X_train, y_train)
rand_forest.score(X_test, y_test)

0.71437834332955097

In [29]:
ids = pd.DataFrame(test['listing_id'])
ids = ids.reset_index(drop = True)
ids.head()

,listing_id
0,7142618
1,7210040
2,7103890
3,7143442
4,6860601


In [30]:
result = rand_forest.predict_proba(test)
result = pd.DataFrame(result)
result.head()

,0,1,2
0,0.06,0.65,0.29
1,0.30,0.21,0.49
2,0.01,0.91,0.08
3,0.04,0.47,0.49
4,0.03,0.81,0.16


In [31]:
submission = pd.concat([ids, result], axis = 1, join_axes = [ids.index])
submission.columns = ['listing_id', 'high', 'medium', 'low']
submission.head()

,listing_id,high,medium,low
0,7142618,0.06,0.65,0.29
1,7210040,0.30,0.21,0.49
2,7103890,0.01,0.91,0.08
3,7143442,0.04,0.47,0.49
4,6860601,0.03,0.81,0.16


In [32]:
submission.to_csv('submission.csv', sep = ',', index = False)

In [100]:
#null accuracy submission

ids['high'] = .077788
ids['medium'] = .227529
ids['low'] = .694683

result = ids
result.head()

,listing_id,high,medium,low
0,7142618,0.077788,0.227529,0.694683
1,7210040,0.077788,0.227529,0.694683
100,7103890,0.077788,0.227529,0.694683
1000,7143442,0.077788,0.227529,0.694683
100000,6860601,0.077788,0.227529,0.694683


In [11]:
result.to_csv('submission.csv', sep = ',', index = False)